In [1]:
rm(list=ls()) # clear workspace
suppressPackageStartupMessages(require(argparse))
suppressPackageStartupMessages(require(tidyverse))
suppressPackageStartupMessages(require(DropletUtils))

In [2]:
run.emptyDrops <- function(m, lower, FDR.thresh=0.01, ...){
  # wrapper for emptyDrops function
  #   m = counts dataframe in genes x cells format
  #   lower = number of total UMI counts below which all barcodes are considered empty droplets
  #   FDR.thresh = threshold above which barcodes are labeled as empty droplets (default 0.01)
  #   ... = additional args to pass to emptyDrops() function
  start.time <- Sys.time()
  
  result <- emptyDrops(m = m, lower = lower, ...)
  
  out <- data.frame(result@listData) %>%
    mutate(empty = as.numeric(FDR > FDR.thresh)) %>%
    mutate(empty = replace_na(empty, 1)) %>%
    mutate(barcode = result@rownames)
  
  print(Sys.time() - start.time)
  return(out)
}

In [3]:
read.counts <- function(path, transpose=F, row.names=NULL){
  start.time <- Sys.time()
  if(str_detect(string = path, pattern = '.csv')){
    counts <- read.csv(path, row.names=row.names)
  }else if(str_detect(string = path, pattern = '.tsv')){
    counts <- read.csv(path, sep = '\t', row.names=row.names)
  }
  
  if(transpose){
    counts <- t(counts)
  }
  print(Sys.time() - start.time) # print file reading time
  return(counts)
}

In [4]:
#library(Seurat)
library(hdf5r)


Attaching package: ‘hdf5r’

The following object is masked from ‘package:SummarizedExperiment’:

    values

The following object is masked from ‘package:GenomicRanges’:

    values

The following object is masked from ‘package:IRanges’:

    values

The following object is masked from ‘package:S4Vectors’:

    values

The following object is masked from ‘package:purrr’:

    flatten_df



In [5]:
#sampleS3 <- read.counts('data/2771-AS-3-ACAGTG_S3.h5ad', transpose=T)

In [6]:
#sampleS3 <- ReadH5AD('data/2771-AS-3-ACAGTG_S3.h5ad')

In [7]:
#library(tidyverse)

In [8]:
#read.table("NonScanpyVersion//Example_dataset.csv.gz", row.names = 1, sep = ",", header = T) -> d

In [9]:
#output <- run.emptyDrops(sampleS3, lower = 500)

In [18]:
data_in <- h5file("../data/colon_10x.h5ad")
data_in_raw <- data_in[['X']][,] #raw count data, make sure this is raw counts/ints

In [19]:
data_in_obs = data_in[['obs']][] #load obs
data_in_var = data_in[['var']][] #load vars
rownames(data_in_raw) = data_in_var$index #set labels
colnames(data_in_raw) = data_in_obs$index #set labels

In [20]:
#data_in_seurat = CreateSeuratObject(data_in_raw,project = 'seurat_obj_name') #create seurat object
#data_in_seurat$batch = data_in_obs$batch #set further labels from h5ad annotations if necessary

In [21]:
#data_in_seurat@assays$RNA@counts

In [22]:
dim(data_in_raw)

[1] 27998  5000

In [23]:
emptyDrops_thresh_output <- run.emptyDrops(data_in_raw, lower = 400)

ERROR: Error in testEmptyDrops(m, lower = lower, ...): no counts available to estimate the ambient profile


In [ ]:
emptyDrops_thresh_output

In [17]:
sum(emptyDrops_thresh_output$empty) # total number of empty drops based on emptyDrops threshold

ERROR: Error in eval(expr, envir, enclos): object 'emptyDrops_thresh_output' not found


In [18]:
write.csv(emptyDrops_thresh_output,"testing/emptyDrops-thresh-colon10x.csv", row.names = FALSE)